In [1]:
import pandas as pd
from geopy.distance import geodesic
from IPython.display import display
import os

if os.path.exists('../data/produtos_cooperativas.csv'):
    print("File exists")
CSV_FILE_PATH = '../data/produtos_cooperativas.csv' # Make sure this file is in the same directory or provide the correct path


File exists


In [2]:
class RecomendadorAgricolaDF:
    def __init__(self, csv_filepath):
        """
        Initializes the recommender system with data from a single CSV file.
        The CSV file should contain cooperative information, their regions,
        lat/lon, and product availability (V/F).
        """
        try:
            df = pd.read_csv(csv_filepath)
        except FileNotFoundError:
            print(f"Error: The file {csv_filepath} was not found.")
            print("Please ensure the CSV_FILE_PATH variable is set correctly.")
            # Create an empty DataFrame with expected columns to allow instantiation but fail gracefully later
            # This helps if the file path is wrong during testing in some environments.
            # A more robust solution might raise an error or handle it differently.
            self.data_exploded = pd.DataFrame(columns=['Nome', 'Região_Original', 'Latitude', 'Longitude', 'Região'])
            self.product_columns = []
            return

        # Identify product columns (assuming they are all columns after 'Longitude')
        try:
            longitude_index = df.columns.get_loc('Longitude')
            self.product_columns = df.columns[longitude_index + 1:].tolist()
        except KeyError:
            print("Error: 'Longitude' column not found in the CSV. Cannot identify product columns.")
            self.data_exploded = pd.DataFrame(columns=['Nome', 'Região_Original', 'Latitude', 'Longitude', 'Região'])
            self.product_columns = []
            return
            
        if not self.product_columns:
            print("Warning: No product columns were identified.")

        # Convert product V/F to True/False
        for col in self.product_columns:
            if col in df.columns:
                df[col] = df[col].replace({'V': True, 'F': False, 'v': True, 'f': False})
            else:
                print(f"Warning: Expected product column '{col}' not found in CSV.")

        # Store original 'Região' if needed later, then split and explode
        df['Região_Original'] = df['Região'] 
        df['Região'] = df['Região'].astype(str).str.split(',') # Ensure it's string, then split
        
        self.data_exploded = df.explode('Região')
        
        # Clean 'Região' names
        self.data_exploded['Região'] = self.data_exploded['Região'].str.strip().str.lower()
        
        # Ensure essential columns are present
        required_cols = ['Nome', 'Latitude', 'Longitude', 'Região']
        if not all(col in self.data_exploded.columns for col in required_cols):
            print(f"Error: The processed data is missing one or more required columns: {required_cols}")
            # Potentially set self.data_exploded to an empty frame with these columns
            # to prevent further errors, or raise an exception.


    def recomendar(self, produtos_alvo, origem, raio_km=20):
        """
        Recommends cooperatives based on target products, origin, and search radius.

        Args:
            produtos_alvo: Lista de produtos (ex: ['Tomate', 'Alface'])
            origem: Tupla (latitude, longitude)
            raio_km: Raio de busca em quilômetros
            
        Returns:
            DataFrame with cooperatives that operate in relevant regions, 
            produce ALL the target products, and are within the specified radius.
        """
        if self.data_exploded.empty:
            print("Recommender not initialized properly or data is empty. Cannot provide recommendations.")
            return pd.DataFrame()
            
        # Validate produtos_alvo are in product_columns
        valid_produtos_alvo = [p for p in produtos_alvo if p in self.product_columns]
        if len(valid_produtos_alvo) != len(produtos_alvo):
            invalid_ones = set(produtos_alvo) - set(valid_produtos_alvo)
            print(f"Warning: The following target products are not in the dataset and will be ignored: {invalid_ones}")
        
        if not valid_produtos_alvo:
            print("No valid target products specified or available in the dataset.")
            return pd.DataFrame()

        # 1. Find "valid regions": regions where at least one cooperative 
        #    produces ALL the target products.
        #    A cooperative entry (after explode) means that cooperative operates in that 'Região'
        #    and its product availability (V/F) is for the cooperative itself.
        coops_com_todos_os_produtos_alvo = self.data_exploded[
            self.data_exploded[valid_produtos_alvo].all(axis=1)
        ]
        
        if coops_com_todos_os_produtos_alvo.empty:
            print(f"No cooperative found that produces all specified products: {valid_produtos_alvo}")
            # Return an empty DataFrame with expected columns
            expected_cols = list(self.data_exploded.columns) + ['Distância (km)']
            return pd.DataFrame(columns=expected_cols)

        nomes_regioes_validas = coops_com_todos_os_produtos_alvo['Região'].unique()

        # 2. Filter to cooperatives that:
        #    a) Operate in one of these `nomes_regioes_validas`.
        #    b) AND themselves produce ALL the `valid_produtos_alvo`.
        #    This effectively means we are just taking `coops_com_todos_os_produtos_alvo`
        #    and ensuring their region is in `nomes_regioes_validas`.
        #    This is already true by definition of `nomes_regioes_validas`.
        #    So, `coops_com_todos_os_produtos_alvo` is our starting point for distance calculation.
        cooperativas_candidatas = coops_com_todos_os_produtos_alvo.copy()
        
        # If after filtering, no candidates, return empty DataFrame
        if cooperativas_candidatas.empty:
            # This case should ideally be caught by the check for coops_com_todos_os_produtos_alvo.empty()
            print("No candidate cooperatives found after initial product and region filtering.")
            expected_cols = list(self.data_exploded.columns) + ['Distância (km)']
            return pd.DataFrame(columns=expected_cols)

        # 3. Calcular distâncias
        #    Make sure Latitude and Longitude are numeric
        cooperativas_candidatas['Latitude'] = pd.to_numeric(cooperativas_candidatas['Latitude'], errors='coerce')
        cooperativas_candidatas['Longitude'] = pd.to_numeric(cooperativas_candidatas['Longitude'], errors='coerce')
        
        # Drop rows with NaN latitude or longitude that might have been coerced
        cooperativas_candidatas.dropna(subset=['Latitude', 'Longitude'], inplace=True)

        if cooperativas_candidatas.empty:
            print("No cooperatives with valid coordinates found.")
            expected_cols = list(self.data_exploded.columns) + ['Distância (km)']
            return pd.DataFrame(columns=expected_cols)
            
        cooperativas_candidatas['Distância (km)'] = cooperativas_candidatas.apply(
            lambda row: int(round(geodesic(origem, (row['Latitude'], row['Longitude'])).km)),
            axis=1
        )
        
        # 4. Filtrar por raio e ordenar
        #    Select relevant columns for the final output to avoid clutter
        #    Keep 'Nome', 'Região' (the specific one for this entry), 'Latitude', 'Longitude', 'Distância (km)'
        #    and the target product columns for verification.
        output_columns = ['Nome', 'Região', 'Latitude', 'Longitude'] + valid_produtos_alvo + ['Distância (km)']
        # Ensure all output columns exist in cooperativas_candidatas before selecting
        output_columns = [col for col in output_columns if col in cooperativas_candidatas.columns]


        resultado_final = cooperativas_candidatas[
            cooperativas_candidatas['Distância (km)'] <= raio_km
        ].sort_values('Distância (km)')
        
        # To avoid duplicate cooperatives if a single cooperative operates in multiple valid regions
        # within the radius, we can drop duplicates based on 'Nome', keeping the one with smallest distance.
        # The sort_values already puts the smallest distance first for each group.
        resultado_final = resultado_final.drop_duplicates(subset=['Nome'], keep='first')

        return resultado_final[output_columns]

In [ ]:
# --- TESTE DE SAÍDA / EXEMPLO DE USO ---
if __name__ == "__main__":
    # Initialize the recommender system with the path to your CSV
    sistema = RecomendadorAgricolaDF(CSV_FILE_PATH)

    # Check if initialization failed (e.g. file not found)
    if sistema.data_exploded.empty and not hasattr(sistema, 'product_columns'):
        print("System initialization failed. Exiting.")
    else:
        print(f"System initialized. Product columns found: {sistema.product_columns[:5]}...") # Print first 5 product columns
        
        # Exemplo: Buscar produtores de Alface e Tomate próximo ao Plano Piloto
        produtos_desejados = ['Alface'] 
        # Check if these products exist in the dataset before searching
        produtos_desejados = [p for p in produtos_desejados if p in sistema.product_columns]

        if not produtos_desejados:
            print("None of the desired products are available in the dataset according to product columns.")
        else:
            print(f"\nSearching for: {produtos_desejados}")
            resultados = sistema.recomendar(
                produtos_alvo=produtos_desejados,
                origem=(-15.794, -47.882),  # Coordenadas do Plano Piloto
                raio_km=300 # Increased radius for more potential results with sample data
            )
            
            print("\n🍅🌱 Top Recomendações:")
            if not resultados.empty:
                display(resultados)
            else:
                print("Nenhuma cooperativa encontrada para os critérios especificados.")
            
            # print("\n🗺️ Mapa Interativo: (Folium map generation would go here if desired)")
            # Example of how you might generate a map if Folium is installed:
            # if not resultados.empty and 'folium' in sys.modules:
            #     mapa = folium.Map(location=origem, zoom_start=10)
            #     folium.Marker(origem, popup="Origem", icon=folium.Icon(color='blue')).add_to(mapa)
            #     for _, row in resultados.iterrows():
            #         folium.Marker(
            #             location=(row['Latitude'], row['Longitude']),
            #             popup=f"{row['Nome']} ({row['Região']}) - {row['Distância (km)']} km",
            #             tooltip=row['Nome']
            #         ).add_to(mapa)
            #     display(mapa)
            # else:
            #     print("Não foi possível gerar o mapa (sem resultados ou Folium não importado).")

# END OF MODIFIED filtro_distancia.ipynb

System initialized. Product columns found: ['Abacate', 'Atemóia', 'Banana', 'Cajamanga', 'Coco']...

Searching for: ['Alface', 'Tomate']

🍅🌱 Top Recomendações:


/tmp/ipykernel_10740/4087946932.py:36: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({'V': True, 'F': False, 'v': True, 'f': False})


,Nome,Região,Latitude,Longitude,Alface,Tomate,Distância (km)
15,"Cooperativa de Serviços Ambientais, Agricultur...",são sebastião,-15.650000,-47.800000,True,True,18
6,Associação dos Produtores do Núcleo Rural Lago...,sobradinho,-15.619109,-47.948728,True,True,21
0,associação de Agricultores Familiares da Eco C...,são sebastião,-15.911175,-47.721429,True,True,22
4,Associação dos Produtores Rurais de Alexandre ...,brazlândia,-15.747628,-48.117130,True,True,26
14,Cooperativa Agrícola da Região de Planaltina (...,ceilândia,-16.014000,-48.000000,True,True,27
16,Cooperativa Agroambiental Palmas do Lago Oeste...,taquara,-15.572147,-48.014295,True,True,28
8,Associação dos Trabalhadores Rurais da Agricul...,sobradinho,-15.542746,-48.029801,True,True,32
12,Cooperativa Agropecuária da Região de Brazlând...,alexandre gusmão,-15.603137,-48.113596,True,True,33
7,Associação dos Produtores Rurais Novo Horizont...,ceilândia,-15.603462,-48.113374,True,True,33
13,Cooperativa de Agricultura Familiar Mista do D...,taquara,-15.765032,-47.493323,True,True,42
